In [1]:
# Dependencies
import pandas as pd
import requests
import numpy as np

In [2]:
#Pull website data and extract table
url = 'https://www.the-numbers.com/movies/franchises'

response = requests.get(url)

tables = pd.read_html(response.text)

f_all_data_df = tables[0]

In [3]:
#Keep relevant rows: Franchise name and Worldwide Box Office
franchise_df = f_all_data_df[["Franchise", "Worldwide Box Office"]]

In [4]:
#Remove currency formatting and drop franchises with no value or 0  
franchise_df['Worldwide Box Office'] = franchise_df['Worldwide Box Office'].str.replace(',', '')
franchise_df['Worldwide Box Office'] = franchise_df['Worldwide Box Office'].str.replace('$', '')
franchise_df['Worldwide Box Office'] = franchise_df['Worldwide Box Office'].astype(float)

franchise_df.replace(0, np.nan, inplace=True)
franchise_df.dropna(inplace=True)

C:\Users\kirst\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\kirst\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kirst\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [5]:
#Export franchise data
#franchise_df.to_csv('franchise_df.csv', index=False)

In [6]:
#Read metadata CSV
csv_file = "Desktop/new_movie_metadata.csv"

metadata_df = pd.read_csv(csv_file, header= 0, encoding= 'unicode_escape', low_memory=False)

In [7]:
#Keep relevant rows: original_title, collection, revenue. Drop rows with 0 value
franchise_single = metadata_df[['original_title', 'belongs_to_collection', 'revenue']]

franchise_single.astype({'revenue': 'float64'}).dtypes

franchise_single = franchise_single.query("revenue > 0")
franchise_single = franchise_single.reset_index(drop=True)
franchise_single.dropna(inplace=True)

In [8]:
#Export data for individual movies
#franchise_single.to_csv('metadata_franchise.csv', index=False)

In [9]:
#Create empty dataframe for final table
final_table = pd.DataFrame (columns = ['Franchise', 'Movie Title','Movie Revenue', 'Total Franchise Revenue', '% of Total Franchise Revenue'])

In [10]:
#Match if collection contains franchise name and input desired data into final table
for idx, fn in franchise_df.iterrows():
    search = fn["Franchise"]
    wwbo = fn["Worldwide Box Office"]
    title_match = franchise_single["belongs_to_collection"].str.contains(search)
    indexes = (title_match[title_match==True].index.values)
    
    revenue = franchise_single.loc[indexes, 'revenue'] 
    movie_title = franchise_single.loc[indexes, 'original_title'] 
    franchise_performance = (revenue / wwbo)*100

    final_data = {'Franchise': search, 'Movie Title': movie_title, 'Movie Revenue': revenue, 'Total Franchise Revenue': wwbo, '% of Total Franchise Revenue': franchise_performance}
    final_table = final_table.append(pd.DataFrame(final_data, columns= ['Franchise', 'Movie Title', 'Movie Revenue', 'Total Franchise Revenue', '% of Total Franchise Revenue']),ignore_index=True)

C:\Users\kirst\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [11]:
#Clean final table
delete='[REC]'

clean_final = final_table[final_table.Franchise != delete]

#Export final table to CSV
clean_final.to_csv('Franchise_Performance_By_Movie.csv', index=False)